***CONECTANDO NO MYSQL***

In [1]:
import mysql.connector

In [2]:
cnx = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "123456"
)

print(cnx)

***CONECTANDO AO SPARK*** 

In [3]:
import os
from pyspark.sql import SparkSession

In [4]:
os.environ["SPARK_HOME"] = "/opt/spark/spark-3.5.5-bin-hadoop3"

In [5]:
import findspark

In [6]:
findspark.init()

In [7]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

25/04/10 21:56:00 WARN Utils: Your hostname, dg-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlp1s0)
25/04/10 21:56:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/10 21:56:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark

In [ ]:
spark.

***LENDO OS DADOS***

In [23]:
path_csv = "../data_raw/csv/"

In [88]:
data_csv = spark.read.csv(path_csv)

In [29]:
data_csv.count()

1344513

***CRIANDO VIEW PARA UTILIZAR O SPARK SQL***

In [105]:
data_csv.createOrReplaceTempView("produtos")

In [91]:
spark.sql("SELECT  * FROM produtos").show(5,False)

+-------------------------------------------------------------------------------------------------+---+---+------+---+---------------+
|_c0                                                                                              |_c1|_c2|_c3   |_c4|_c5            |
+-------------------------------------------------------------------------------------------------+---+---+------+---+---------------+
|Adaptador Universal Viagem + Mala Armor - Gshield                                                |0.0|0  |564.0 |0.0|Malas de Viagem|
|Mala De Viagem Grande Bolsa De Mao Azul Transversal Resistente Poliéster Esportiva Tiracolo Yins |3.0|1  |128.03|0.0|Malas de Viagem|
|Mala de Viagem de Bordo Impermeável Até 15kg Flexível (Preto)                                    |4.0|1  |159.99|0.0|Malas de Viagem|
|Bagagem infantil para meninos, mala de rodas giratórias de dinossauro fofo para crianças pequenas|4.6|102|319.99|0.0|Malas de Viagem|
|MALA DE BORDO PEQUENA PARA VIAGEM EM ABS YINS YS21110 

***MUDANDO NOMES DAS COLUNAS***

In [92]:
from pyspark.sql  import functions as f

In [101]:
produtos_col_names = ["TITULO","NOTA","NUMERO_AVALIACOES","PRECO","LISTA_PREÇO","CATEGORIA"]

In [102]:
for index,colNames in enumerate(produtos_col_names):
    data_csv = data_csv.withColumnRenamed(f"_c{index}",colNames)

In [103]:
data_csv.columns

['TITULO', 'NOTA', 'NUMERO_AVALIACOES', 'PRECO', 'LISTA_PREÇO', 'CATEGORIA']

In [106]:
spark.sql("SELECT * FROM produtos").show(20,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------+-----------+---------------+
|TITULO                                                                                                                                                                            |NOTA|NUMERO_AVALIACOES|PRECO |LISTA_PREÇO|CATEGORIA      |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------+-----------+---------------+
|Adaptador Universal Viagem + Mala Armor - Gshield                                                                                                                                 |0.0 |0                |564.0 |0.0        |Malas de Viagem|
|Mala De Viagem Grande Bolsa De Mao Azul Tra

***VERIFICANDO O CONTEUDO DAS COLUNAS***

In [107]:
spark.sql("SELECT DISTINCT CATEGORIA FROM produtos ORDER BY CATEGORIA").show(truncate = False)

+-----------------------------------------+
|CATEGORIA                                |
+-----------------------------------------+
|Acessórios de Ferramentas Elétricas      |
|Acessórios e Artigos Eletrônicos         |
|Acessórios e Peças para Motos            |
|Acessórios para Celular                  |
|Acessórios para Computador               |
|Acessórios para Gatos                    |
|Acessórios para Home Theater             |
|Acessórios para Viagem                   |
|Achados em Moda                          |
|Alimentação de Bebês e Crianças Pequenas |
|Alimentos e Bebidas                      |
|Apps e Jogos                             |
|Ar e Ventilação                          |
|Assadeiras, Fôrmas e Recipientes de Forno|
|Audiolivros Audible                      |
|Automotivo                               |
|Banho, Higiene e Troca de Fraldas do Bebê|
|Bebidas Alcoólicas                       |
|Bebês                                    |
|Beleza                         

***SEPARANDO POR CATEGORIA***

In [108]:
data_app_and_jogos = spark.sql("SELECT * FROM produtos WHERE CATEGORIA = 'Apps e Jogos' ")

In [116]:
data_app_and_jogos.count()

15913

In [110]:
data_app_and_jogos.createOrReplaceTempView("view_produtos_jogos_and_apps")

In [111]:
spark.sql("SELECT * FROM view_produtos_jogos_and_apps").show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------+-----------+------------+
|TITULO                                                                                                                           |NOTA|NUMERO_AVALIACOES|PRECO |LISTA_PREÇO|CATEGORIA   |
+---------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------+-----------+------------+
|Snake Run                                                                                                                        |4.3 |314              |0.0   |0.0        |Apps e Jogos|
|Claro tv+                                                                                                                        |2.5 |222              |0.0   |0.0        |Apps e Jogos|
|NBA on Fire TV                                                  

In [117]:
spark.sql("SELECT * FROM view_produtos_jogos_and_apps WHERE PRECO > 0 ").show()

+--------------------+----+-----------------+------+-----------+------------+
|              TITULO|NOTA|NUMERO_AVALIACOES| PRECO|LISTA_PREÇO|   CATEGORIA|
+--------------------+----+-----------------+------+-----------+------------+
|Minecraft – Pocke...| 4.2|           137007|  20.0|        0.0|Apps e Jogos|
|Volume Control fo...| 4.2|              723|  7.99|        0.0|Apps e Jogos|
|PK XD : Building ...| 3.6|               88| 96.43|        0.0|Apps e Jogos|
|PK XD : Block Cra...| 3.8|               34|100.79|        0.0|Apps e Jogos|
|[3D Platformer] S...| 3.9|               32|104.33|        0.0|Apps e Jogos|
|   Color Dancing Hop| 3.9|               44| 19.52|        0.0|Apps e Jogos|
|     Baby Panda Care| 3.7|              110|  42.1|        0.0|Apps e Jogos|
|Frozen ASMR: Hone...| 3.0|               34| 40.26|        0.0|Apps e Jogos|
|  School Party Craft| 3.8|              203| 15.35|        0.0|Apps e Jogos|
|    Scary Teacher 3D| 3.5|             7726|  4.02|        0.0|

In [118]:
spark.sql("SELECT * FROM view_produtos_jogos_and_apps WHERE PRECO > 0 ").count()

1074